# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    orden::Int64 #el orden de la serie de taylor.
    coff::Array{T,1}   #coeficientes en la serie de taylor.
end

Taylor

In [6]:
function Taylor(orden,coff)
    return Taylor(promote(orden,coff)...)#al igual que la tarea3, se bhace la especificación de cómo estará compuesto la función taylor
end
function taylor(n,v)    #ESTE AGREGADO ES NECESARIO PARA GARANTIZAR QUE TODOS LOS ELEMENTOS INGRESADOS EN EL VECTOR SEAN DEL MISMO ORDEN QUE LA N PROPUESTA
    if length(v)==n
        return Taylor(n,v)
    else
        println("función no válida")
    end
end

taylor (generic function with 1 method)

{Main.Taylor})(Any, Any) in module Main at In[5]:2 overwritten at In[6]:2.


In [7]:
a=[2,3,4,5,1.2]#es necesario recordar que sólo reconoce vectores con aunque sea un valor de punto flotante

5-element Array{Float64,1}:
 2.0
 3.0
 4.0
 5.0
 1.2

In [8]:
a[4]

5.0

In [9]:
taylor(2,a)

función no válida


In [10]:
a=[2,4]

2-element Array{Int64,1}:
 2
 4

In [11]:
taylor(2,a)

Taylor{Int64}(2,[2,4])

Esto sólo es un ejemplo para ver que el nuevo type *Taylor* funciona correctamente

In [12]:
function xTaylor(n)
    w=zeros(n) #genera un vector de n ceros
    w=deleteat!(w,n)#el último elemento no es cero
    w=append!(w,1.0) #el último elemento del vector de orden cero es 1.0
    return taylor(n,w)
end

xTaylor (generic function with 1 method)

In [13]:
xTaylor(2)

Taylor{Float64}(2,[0.0,1.0])

In [14]:
m=[1,2]

2-element Array{Int64,1}:
 1
 2

In [16]:
taylor(2,m)

Taylor{Int64}(2,[1,2])

In [18]:
n=ans

Taylor{Int64}(2,[1,2])

In [19]:
n.coff

2-element Array{Int64,1}:
 1
 2

In [20]:
q=taylor(2,[1,5])

Taylor{Int64}(2,[1,5])

In [21]:
n.coff+q.coff

2-element Array{Int64,1}:
 2
 7

In [22]:
size(n.coff)==size(q.coff)

true

In [23]:
function multvector(a,b)  #multiplicación entre vectores como lo describen los notebooks
    n=a.orden    #el orden de taylor
    elemento=0.0
    k=2      #número máximo de coeficientes
    v=zeros(n)  #genero un vector de grado n lleno de ceros
    if a.orden==b.orden    #muy importante, no existe producto sin esa condición
        println("operación válida")
        for i in 1:n    #el ciclo for se hará desde cero hasta el orden n
            elemento=elemento+(a.coff[k-1])*(b.coff[k-i])   #suma los elementos de los productos
            if k <= n-1 #si el valor máximo es menor a n-1, se incrementa por 1
            k+=1
            end 
            v[i]=elemento
        end
        return taylor(n,v)
        else println("operación no válida")   #si los grados son distintos entre si
    end
end

multvector (generic function with 1 method)

In [24]:
a=taylor(2,[1,2])

Taylor{Int64}(2,[1,2])

In [25]:
b=taylor(2,[7,90])

Taylor{Int64}(2,[7,90])

In [26]:
multvector(a,b)

operación válida


LoadError: BoundsError: attempt to access 2-element Array{Int64,1} at index [0]

In [28]:
length(1)

1

In [30]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función
+(A::Taylor,B::Taylor)=Taylor(A.orden,A.coff+B.coff)
-(A::Taylor,B::Taylor)=Taylor(A.orden,A.coff-B.coff)

*(A::Real,B::Taylor)=Taylor(B.orden,A*B.coff)
*(A::Taylor,B::Real)=Taylor(A.orden,B*A.coff)

/(A::Taylor,B::Real)=Taylor(A.orden,(A.coff)/B)

==(A::Taylor,B::Taylor)=(A.orden==B.orden && A.coff==B.coff)

== (generic function with 119 methods)

In [31]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("runtest_taylor.jl")

Test Summary:      | Pass  Total
  pruebas realizadas |    6      6


Base.Test.DefaultTestSet("pruebas realizadas",Any[Test Passed
  Expression: Taylor(2,[1,2]) + Taylor(2,[5,6]) == Taylor(2,[6,8])
   Evaluated: Taylor{Int64}(2,[6,8]) == Taylor{Int64}(2,[6,8]),Test Passed
  Expression: Taylor(3,[3,2,1]) - Taylor(3,[3,2,1]) == Taylor(3,[0,0,0])
   Evaluated: Taylor{Int64}(3,[0,0,0]) == Taylor{Int64}(3,[0,0,0]),Test Passed
  Expression: Taylor(3,[10,20,30]) * 3 == Taylor(3,[30,60,90])
   Evaluated: Taylor{Int64}(3,[30,60,90]) == Taylor{Int64}(3,[30,60,90]),Test Passed
  Expression: 3 * Taylor(3,[10,20,30]) == Taylor(3,[30,60,90])
   Evaluated: Taylor{Int64}(3,[30,60,90]) == Taylor{Int64}(3,[30,60,90]),Test Passed
  Expression: Taylor(4,[2,4,6,8]) / 2 == Taylor(4,[1,2,3,4])
   Evaluated: Taylor{Float64}(4,[1.0,2.0,3.0,4.0]) == Taylor{Int64}(4,[1,2,3,4]),Test Passed
  Expression: Taylor(2,[1,2]) == Taylor(2,[1,2])
   Evaluated: Taylor{Int64}(2,[1,2]) == Taylor{Int64}(2,[1,2])],false)

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.